In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('E:/百度/train+cnn.txt', encoding='utf8') as file:
    line_list1 = [k.strip() for k in file.readlines()]
    #读取每行
    train_label_list = [k.split()[0] for k in line_list1]
    #将标签依次取出
    train_content_list = [k.split(maxsplit=1)[1] for k in line_list1]
    #将内容依次取出,此处注意split()选择最大分割次数为1,否则句子被打断.
with open('E:/百度/test+cnn.txt', encoding='utf8') as file:
    line_list2 = [k.strip() for k in file.readlines()]
    test_label_list = [k.split()[0] for k in line_list2]
    test_content_list = [k.split(maxsplit=1)[1] for k in line_list2]
with open('E:/百度/val+cnn.txt', encoding='utf8') as file:
    line_list3 = [k.strip() for k in file.readlines()]
    val_label_list = [k.split()[0] for k in line_list3]
    val_content_list = [k.split(maxsplit=1)[1] for k in line_list3]

In [3]:
from keras.utils import plot_model
from keras.layers import Embedding
import gensim
from gensim.models.word2vec import Word2Vec,LineSentence,Text8Corpus

Using TensorFlow backend.
C:\Users\hkdw235\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
model = Word2Vec.load('C:/Users/hkdw235/PycharmProjects/untitled/THUCNews/cont.model')

In [5]:
word_vectors = model.wv

In [30]:
EMBEDDING_DIM = 128 # 词向量空间维度
num_classes=10 # 类型数量
maxlen=600 # 文本长度
max_words=10000 # 词汇表数量
kernel=8
filters=128

In [7]:
embeddings_index = {}
for word, vocab_obj in model.wv.vocab.items():
    if int(vocab_obj.index) < max_words:
        embeddings_index[word] = word_vectors[word]
del model, word_vectors # 删掉gensim模型释放内存
print('Found %s word vectors.' % len(embeddings_index))

Found 10000 word vectors.


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [9]:
tokenizer = Tokenizer(num_words=max_words) # 传入我们词向量的字典
content_list=train_content_list+test_content_list+val_content_list
tokenizer.fit_on_texts(content_list) # 传入我们的训练数据，得到训练数据中出现的词的字典
sequences1 = tokenizer.texts_to_sequences(train_content_list) # 根据训练数据中出现的词的字典，将训练数据转换为sequences
sequences2 = tokenizer.texts_to_sequences(test_content_list) # 根据训练数据中出现的词的字典，将训练数据转换为sequences
sequences3 = tokenizer.texts_to_sequences(val_content_list) # 根据训练数据中出现的词的字典，将训练数据转换为sequences
word_index = tokenizer.word_index
print('Found %s unique tokens2.' % len(word_index))

Found 383707 unique tokens2.


In [10]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
train_Y = to_categorical(label.fit_transform(train_label_list),num_classes=num_classes)
test_Y = to_categorical(label.fit_transform(test_label_list),num_classes=num_classes)
val_Y = to_categorical(label.fit_transform(val_label_list),num_classes=num_classes)

In [11]:
train_X = pad_sequences(sequences1,maxlen=maxlen,truncating='post',padding='post') # 和原作者的不同
test_X = pad_sequences(sequences2,maxlen=maxlen,truncating='post',padding='post') # 和原作者不同 
val_X = pad_sequences(sequences3,maxlen=maxlen,truncating='post',padding='post') # 和原作者不同 

In [12]:
from keras.models import Model
from keras import layers
from keras.initializers import he_normal


In [13]:
embedding_matrix = np.zeros((max_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # 文本数据中的词在词向量字典中没有，向量为取0；如果有则取词向量中该词的向量
        embedding_matrix[i] = embedding_vector

In [16]:
import numpy as np
index1=np.arange(len(train_Y))
np.random.shuffle(index1)
index2=np.arange(len(test_Y))
np.random.shuffle(index2)
index3=np.arange(len(val_Y))
np.random.shuffle(index3)
train_X = train_X[index1]
train_Y = train_Y[index1]
test_X = test_X[index2]
test_Y = test_Y[index2]
val_X = val_X[index3]
val_Y = val_Y[index3]

In [32]:
input = layers.Input(shape=(maxlen,),dtype='int32')
embedding = layers.Embedding(max_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)(input)

convs = []
filter_sizes = [2,3,4]
for fsz in filter_sizes:
    conv1 = layers.Conv1D(filters,kernel_size=fsz,activation='relu')(embedding)
    pool1 = layers.MaxPooling1D(maxlen-fsz+1)(conv1)
    pool1 = layers.Flatten()(pool1)
    convs.append(pool1)
merge = layers.concatenate(convs,axis=1)
dense1=layers.Dense(64,activation='relu',kernel_initializer='glorot_normal')(merge)
dropout = layers.Dropout(0.2)(dense1)
predict_y = layers.Dense(10,activation='softmax')(dropout)
model = Model(inputs=input,outputs=predict_y)
from keras import optimizers
optimizer = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['categorical_accuracy'] )
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 600)          0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 600, 128)     1280000     input_11[0][0]                   
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 599, 128)     32896       embedding_11[0][0]               
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 598, 128)     49280       embedding_11[0][0]               
__________________________________________________________________________________________________
conv1d_12 

In [33]:
from keras import callbacks
reduce=callbacks.ReduceLROnPlateau(monitor='categorical_crossentropy',factor=0.2,min_lr=0.1)
Tsbord=callbacks.TensorBoard(batch_size=64)
model.fit(train_X,train_Y,batch_size=64,
          epochs=5,verbose=1,
          validation_data=(val_X,val_Y),
          callbacks=[reduce,Tsbord])

Train on 50000 samples, validate on 5000 samples
Epoch 1/5
50000/50000 [==============================] - 429s 9ms/step - loss: 0.2488 - categorical_accuracy: 0.9291 - val_loss: 0.1020 - val_categorical_accuracy: 0.9712
Epoch 2/5


C:\Users\hkdw235\Anaconda3\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `categorical_crossentropy` which is not available. Available metrics are: val_loss,val_categorical_accuracy,loss,categorical_accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


50000/50000 [==============================] - 429s 9ms/step - loss: 0.0905 - categorical_accuracy: 0.9730 - val_loss: 0.1075 - val_categorical_accuracy: 0.9690
Epoch 3/5
50000/50000 [==============================] - 409s 8ms/step - loss: 0.0583 - categorical_accuracy: 0.9826 - val_loss: 0.0871 - val_categorical_accuracy: 0.9754
Epoch 4/5
50000/50000 [==============================] - 399s 8ms/step - loss: 0.0403 - categorical_accuracy: 0.9874 - val_loss: 0.1148 - val_categorical_accuracy: 0.9646
Epoch 5/5
50000/50000 [==============================] - 409s 8ms/step - loss: 0.0260 - categorical_accuracy: 0.9920 - val_loss: 0.1032 - val_categorical_accuracy: 0.9714


In [40]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import mglearn
import matplotlib.pyplot as plt
def predictAll(test_X, batch_size=100):
    predict_value_list = []
    for i in range(0, len(test_X), batch_size):
        X = test_X[i: i + batch_size]
        predict_value = model.predict(X)
        predict_value_list.extend(predict_value)
    return np.array(predict_value_list)

Y = predictAll(test_X)
y = np.argmax(Y, axis=1)
score_image = mglearn.tools.heatmap(confusion_matrix(test_Y, y),
                                    xlabel='Predicted label',ylabel='True label',xticklabels=label.classes_,
                                    yticklabels=label.classes_,cmap=plt.cm.gray_r,fmt='%d')
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.title('混淆矩阵')
plt.gca().invert_yaxis()
plt.show()

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def eval_model(y_true, y_pred, labels):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred)
    # 计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [tot_p],
        u'Recall': [tot_r],
        u'F1': [tot_f1],
        u'Support': [tot_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

eval_model(test_label_list, predict_label_list, label.classes_)